_Lambda School Data Science_

# Join datasets

Objectives
- concatenate data with pandas
- merge data with pandas

Links
- [Pandas Cheat Sheet](https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf)
  - Combine Data Sets: Standard Joins
- Python Data Science Handbook
  - [Chapter 3.6](https://jakevdp.github.io/PythonDataScienceHandbook/03.06-concat-and-append.html), Combining Datasets: Concat and Append
  - [Chapter 3.7](https://jakevdp.github.io/PythonDataScienceHandbook/03.07-merge-and-join.html), Combining Datasets: Merge and Join

## Download data

We’ll work with a dataset of [3 Million Instacart Orders, Open Sourced](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2)!

In [5]:
!wget https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz

--2019-01-15 18:39:22--  https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.84.165
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.84.165|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 205548478 (196M) [application/x-gzip]
Saving to: ‘instacart_online_grocery_shopping_2017_05_01.tar.gz.1’

instacart_online_gr 100%[===================>] 196.03M  45.6MB/s    in 4.7s    

2019-01-15 18:39:27 (41.6 MB/s) - ‘instacart_online_grocery_shopping_2017_05_01.tar.gz.1’ saved [205548478/205548478]



In [6]:
!tar --gunzip --extract --verbose --file=instacart_online_grocery_shopping_2017_05_01.tar.gz

instacart_2017_05_01/
instacart_2017_05_01/._aisles.csv
instacart_2017_05_01/aisles.csv
instacart_2017_05_01/._departments.csv
instacart_2017_05_01/departments.csv
instacart_2017_05_01/._order_products__prior.csv
instacart_2017_05_01/order_products__prior.csv
instacart_2017_05_01/._order_products__train.csv
instacart_2017_05_01/order_products__train.csv
instacart_2017_05_01/._orders.csv
instacart_2017_05_01/orders.csv
instacart_2017_05_01/._products.csv
instacart_2017_05_01/products.csv


In [7]:
%cd instacart_2017_05_01

/content/instacart_2017_05_01


## Goal: Reproduce this example

The first two orders for user id 1:

In [8]:
from IPython.display import display, Image
url = 'https://cdn-images-1.medium.com/max/1600/1*vYGFQCafJtGBBX5mbl0xyw.png'
example = Image(url=url, width=600)

display(example)

## Load data

Here's a list of all six CSV filenames

In [9]:
!ls -lh

total 681M
-rw-r--r-- 1 502 staff 2.6K May  2  2017 aisles.csv
-rw-r--r-- 1 502 staff  270 May  2  2017 departments.csv
-rw-r--r-- 1 502 staff 551M May  2  2017 order_products__prior.csv
-rw-r--r-- 1 502 staff  24M May  2  2017 order_products__train.csv
-rw-r--r-- 1 502 staff 104M May  2  2017 orders.csv
-rw-r--r-- 1 502 staff 2.1M May  2  2017 products.csv


For each CSV
- Load it with pandas
- Look at the dataframe's shape
- Look at its head (first rows)
- `display(example)`
- Which columns does it have in common with the example we want to reproduce?

### aisles

In [0]:
import pandas as pd

aisles = pd.read_csv('aisles.csv')

In [0]:
# aisles does not have any columns we need

### departments

In [0]:
depts_df = pd.read_csv('departments.csv')

In [13]:
depts_df.sample(5)

,department_id,department
12,13,pantry
6,7,beverages
2,3,bakery
17,18,babies
7,8,pets


In [0]:
# departments does not have any columns we need

### order_products__prior

In [0]:
order_products_prior_df = pd.read_csv('order_products__prior.csv')

In [16]:
order_products_prior_df.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [0]:
# has order_id	product_id	add_to_cart_order

### order_products__train

In [0]:
order_products_train_df = pd.read_csv('order_products__train.csv')

In [19]:
order_products_train_df.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [0]:
# also has order_id	product_id	add_to_cart_order

### orders

In [0]:
orders_df = pd.read_csv('orders.csv')

In [22]:
orders_df.shape

(3421083, 7)

In [23]:
orders_df.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


### products

In [0]:
products_df = pd.read_csv('products.csv')

In [25]:
products_df.shape

(49688, 4)

In [26]:
products_df.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [0]:
# has product_id product_name

## Concatenate order_products__prior and order_products__train

In [0]:
order_products_df = pd.concat([order_products_prior_df, order_products_train_df])

In [29]:
order_products_df.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


## Get a subset of orders — the first two orders for user id 1

In [30]:
sub_df = (orders_df.loc[(orders_df.user_id == 1) &
                        (orders_df.order_number <= 2)]
                   .sort_values('order_number'))

sub_df

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0


## Merge dataframes

In [39]:
display(example)

In [0]:
orders_cols = ['user_id', 'order_id', 'order_number', 'order_dow', 'order_hour_of_day']
order_products_cols = ['order_id', 'add_to_cart_order', 'product_id']

df = (sub_df[cols].merge(order_products_df[order_products_cols],
                            how='left', on='order_id')
                  .merge(products_df[['product_id', 'product_name']], 
                            how='left', on='product_id')
                  .sort_values(['user_id', 'order_number']))

In [32]:
df

,user_id,order_id,order_number,order_dow,order_hour_of_day,add_to_cart_order,product_id,product_name
0,1,2539329,1,2,8,1.0,196,Soda
1,1,2539329,1,2,8,2.0,14084,Organic Unsweetened Vanilla Almond Milk
2,1,2539329,1,2,8,3.0,12427,Original Beef Jerky
3,1,2539329,1,2,8,4.0,26088,Aged White Cheddar Popcorn
4,1,2539329,1,2,8,5.0,26405,XL Pick-A-Size Paper Towel Rolls
5,1,2398795,2,3,7,1.0,196,Soda
6,1,2398795,2,3,7,2.0,10258,Pistachios
7,1,2398795,2,3,7,3.0,12427,Original Beef Jerky
8,1,2398795,2,3,7,4.0,13176,Bag of Organic Bananas
9,1,2398795,2,3,7,5.0,26088,Aged White Cheddar Popcorn


# Assignment

These are the top 10 most frequently ordered products. How many times was each ordered? 

1. Banana
2. Bag of Organic Bananas
3. Organic Strawberries
4. Organic Baby Spinach 
5. Organic Hass Avocado
6. Organic Avocado
7. Large Lemon 
8. Strawberries
9. Limes 
10. Organic Whole Milk

First, write down which columns you need and which dataframes have them.

Next, merge these into a single dataframe.

Then, use pandas functions from the previous lesson to get the counts of the top 10 most frequently ordered products.

## Stretch challenge

The [Instacart blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2) has a visualization of "**Popular products** purchased earliest in the day (green) and latest in the day (red)." 

The post says,

> "We can also see the time of day that users purchase specific products.

> Healthier snacks and staples tend to be purchased earlier in the day, whereas ice cream (especially Half Baked and The Tonight Dough) are far more popular when customers are ordering in the evening.

> **In fact, of the top 25 latest ordered products, the first 24 are ice cream! The last one, of course, is a frozen pizza.**"

Your challenge is to reproduce the list of the top 25 latest ordered popular products.

We'll define "popular products" as products with more than 2,900 orders.

In [0]:
orders_cols = ['user_id', 'order_id', 'order_number', 'order_dow', 'order_hour_of_day']
order_products_cols = ['order_id', 'add_to_cart_order', 'product_id']

df = (orders_df[cols].merge(order_products_df[order_products_cols],
                            how='left', on='order_id')
                     .merge(products_df[['product_id', 'product_name']], 
                            how='left', on='product_id')
                     .sort_values(['user_id', 'order_number']))

In [36]:
df.product_name.value_counts()[0:10]

Banana                    491291
Bag of Organic Bananas    394930
Organic Strawberries      275577
Organic Baby Spinach      251705
Organic Hass Avocado      220877
Organic Avocado           184224
Large Lemon               160792
Strawberries              149445
Limes                     146660
Organic Whole Milk        142813
Name: product_name, dtype: int64

In [0]:
descripts_df = (df.groupby('product_name')
                  .order_hour_of_day.agg(['mean', 'count']))
                  #.loc[df['count'] > 2900])

In [59]:
(descripts_df.loc[descripts_df['count'] > 2900]
             .sort_values('mean', ascending=False)
             .iloc[:25])

,mean,count
product_name,,
Half Baked Frozen Yogurt,14.526986,2983
The Tonight Dough™ Ice Cream,14.526299,3137
Americone Dream® Ice Cream,14.394912,6447
Half Baked® Ice Cream,14.388942,6222
Phish Food® Ice Cream,14.373308,3177
Milk and Cookies Ice Cream,14.369711,5223
Sicilian Pistachio,14.306421,3146
Chocolate Peanut Butter Cup Gelato,14.303480,4511
Birthday Cake Light Ice Cream,14.292018,5462
